* 자신만의 손실 함수, 지표, 층, 모델, 초기화, 규제, 가중치 규제 등을 만들어 세부적으로 제어하고 싶을 때 저수준 파이썬 API 필요함.
* 이 장에서 그걸 보겠다.

## 12.1 텐서플로 훑어보기

* 대규모 머신러닝에 잘 맞도록 튜닝되어 있음. 강력한 수치 계산용 라이브러리
* 구글에서 만들었으며 현재 가장 인기 있는 딥러닝 라이브러리
* JIT 컴파일러 제공: JIT 컴파일러(동적 번역)란 ... -> 계산 최적화로 메모리 사용량을 줄이고 속도를 높인다.
*  계산 그래프는 플랫폼에 중립적인 포맷으로 내보낼 수 있으므로 한 환경(예컨대 리눅스에 있는 파이썬)에서 TF 모델로 훈련하고 다른 환경(안드로이드 장치에 있는 자바)에서 실행할 수 있다.
* 자동 미분 기능과 고성능 옵티마이저를 제공하므로 모든 종류의 손실 함수를 쉽게 최소화할 수 있다.

* 저수준의 TF 연산(op)은 C++ 코드로 구현되어 있다.
* 연산은 커널(kernel)이라 불리는 여러 구현을 가지는데 각 커널은 CPU, GPU, TPU 같은 특정 장치에 맞추어 만들어졌다.

## 12.2 넘파이처럼 텐서플로 사용하기
* 텐서는 다차원 배열을 의미한다. 스칼라 값도 가질 수 있음.
* 사용자 정의 손실 함수, 사용자 정의 지표, 사용자 정의 층 등을 만들 때 텐서 중요함.

### 12.2.1 텐서와 연산

* TF는 넘파이와 비슷하게 작동
* 넘파이에서 볼 수 있는 대부분의 연산을 제공하지만 일부 함수들은 이름이 다름.

In [2]:
import tensorflow as tf
tf.constant([[1., 2., 3.], [4., 5., 6.]]) # 행렬

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [3]:
tf.constant(42)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [4]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
t.shape

TensorShape([2, 3])

In [5]:
t.dtype

tf.float32

In [6]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [7]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

* 모든 종류의 텐서 연산이 가능하다.

In [8]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [9]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [10]:
t @ tf.transpose(t) # @는 행렬의 곱셈. Transpose는 전치

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

### 12.2.2 텐서와 넘파이

In [11]:
import numpy as np
a = np.array([2., 4., 5.])
tf.constant(a)
t.numpy()
tf.square(a)
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

### 12.2.3 타입 변환
* 타입 변환은 성능을 감소시킬 수 있음.
* 실수 텐서와 정수 텐서는 더할 수 없다. 32비트 실수와 64비트 실수도 더할 수 없다.

In [12]:
tf.constant(2.) + tf.constant(40)

InvalidArgumentError: ignored

In [ ]:
tf.constant(2.) + tf.constant(40., dtype = tf.float64)

### 12.2.4 변수

## 12.3 사용자 정의 모델과 훈련 알고리즘

### 12.3.1 사용자 정의 손실 함수
* 회귀 모델을 훈련하는 데 훈련 세트에 잡음 데이터가 있다고 가정하자. 후버 손실을 사용하면 좋다. 후버 손실은 아직 공식 케라스 API에서 지원하지 않으므로 직접 구현해보자.

In [13]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

### 12.3.2 사용자 정의 요소를 가진 모델을 저장하고 로드하기
* 사용자 정의 객체를 포함한 모델을 로드할 땐 그 이름과 객체를 매핑해야 한다.